In [6]:
import pprint
import requests
import pprint

leagueID = "2132617421"
#leagueID = "2141576138"

view = "mDraftDetail"

#URL = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/segments/0/leagues/" + leagueID + "?view=mTeam"
#URL = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/segments/0/leagues/" + leagueID + "?view=mRoster"
#URL = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/segments/0/leagues/" + leagueID + "?view=mDraftDetail"
#URL = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/segments/0/leagues/" + leagueID + "?view=mMatchup"

URL = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/segments/0/leagues/" + leagueID + "?view="+view


r = requests.get(URL,
                cookies={"espn_s2": "AEBlmW%2BnPlQPu68%2B3lzmpbsb5hNTRKEtHrL%2Bq8FkioyJ%2BAkHtJm7yYEagGmtwT%2BH4Wb9lhRdwJzDvYyDuUQ8huceOylAFUJtO3MVkxn0EsDJk47eyalQUduQyirWQiRoByYOp%2Bjhc8pG4Gfi2KftR2%2FBXgElrd734Lz0TBXCYDb33tB3PQt8S1Q6GcdZZ757%2BMeBz%2Fd87XfRCkn3Kgs%2FnB40EJtzL30SlEPeKBmB%2ByUdrysHbYZz9ESlKSIkBuey92sudtrZvPf5NUAIoAOmX2Py7fM1V1E%2F0zHImDLRUcar%2Bw%3D%3D",
                         "swid": "{9AED192A-A64B-4ECA-9772-BDDE2396067C}"})
data = r.json()

reader = pprint.PrettyPrinter()
reader.pprint(data)
# for player in data['players']:
#     reader.pprint(player['player']['fullName'])

{'draftDetail': {'drafted': False,
                 'inProgress': False,
                 'picks': [{'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 1,
                            'keeper': False,
                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNumber': 1,
                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 1,
                            'roundPickNumber': 1,
                            'teamId': 1,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 2,
                            'keeper': False,
                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNum

                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNumber': 48,
                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 3,
                            'roundPickNumber': 16,
                            'teamId': 16,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 49,
                            'keeper': False,
                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNumber': 49,
                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 4,
                            'roundPickNumber': 1,
                            'teamId': 16,
 

                            'roundId': 6,
                            'roundPickNumber': 8,
                            'teamId': 9,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 89,
                            'keeper': False,
                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNumber': 89,
                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 6,
                            'roundPickNumber': 9,
                            'teamId': 8,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 90,
                            'keeper': False,
                           

                            'roundId': 8,
                            'roundPickNumber': 15,
                            'teamId': 2,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 128,
                            'keeper': False,
                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNumber': 128,
                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 8,
                            'roundPickNumber': 16,
                            'teamId': 1,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 129,
                            'keeper': False,
                      

                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 11,
                            'roundPickNumber': 2,
                            'teamId': 2,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 163,
                            'keeper': False,
                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNumber': 163,
                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 11,
                            'roundPickNumber': 3,
                            'teamId': 3,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
      

                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 14,
                            'roundPickNumber': 9,
                            'teamId': 8,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
                            'id': 218,
                            'keeper': False,
                            'lineupSlotId': 0,
                            'nominatingTeamId': 0,
                            'overallPickNumber': 218,
                            'playerId': -1,
                            'reservedForKeeper': False,
                            'roundId': 14,
                            'roundPickNumber': 10,
                            'teamId': 7,
                            'tradeLocked': False},
                           {'autoDraftTypeId': 0,
                            'bidAmount': 0,
     